In [7]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
from IPython.display import HTML
from matplotlib import animation, rc
from scipy.integrate import odeint  
import configparser

In [2]:
# time parameters
dt = 0.005
tLast = 2 
n_frames = int(tLast/dt)
tInt = np.arange(0, tLast+dt, dt)

# animation parameters
fps = 100 

# simulation parameters
n = 5 # number of pendulums 
g = -10  # gravitational constant [m/s^2]
b = 0.01  # damping term 
m = 1* np.ones((n ,1) ) / n  # mass of all the pendulum bulbs
l_vec = np.ones((n ,1) ) /n  # length of pendulum beams

# initial conditions (random position currently) 
y0 = np.zeros(n*2) 
y0[:n] = np.random.rand(n)*np.pi   

# plotting parameters
fig_h = 7
fig_v = 4
window = 1.1
dot_scale = 500

In [3]:
def pendulum_dynamics(y,t): 
    # define pendulum dynamics 
    dydt =  np.zeros([2*n])  
    M_ij = np.zeros((n,n) )
    C_ij = np.zeros((n,n) )
    S_ij = np.zeros((n,n) ) 
    theta = y[ 0:n]  
    for i in range(n ):
        for j in range(n ): 
            m_max = np.max([i,j])  
            M_ij[i,j] = np.sum(m[m_max:])
            S_ij[i,j] = np.sin(theta[ i]-theta[ j])
            C_ij[i,j] = np.cos(theta[ i]-theta[ j])
    L = np.multiply(l_vec,l_vec.transpose()) 
    M = np.multiply(M_ij, np.multiply( L,C_ij)  )  
    C = np.multiply(M_ij, np.multiply( L,S_ij)  )  
    np.fill_diagonal(C,0)
    K = np.multiply( np.diag(l_vec)*g , M_ij) 
    D = np.diag(np.ones(n))*b 
    dydt[0:n] = y[n:n*2]
    dydt[ n:n*2] = - np.dot(np.linalg.inv(M) , (    np.dot(C,y[ n:n*2]**2) +      # 
                                                np.dot( K, np.sin( y[0:n]) ) +   # 
                                                np.dot(D,y[n:n*2])  )    ) 
    return dydt

def init():
    # for animation: initialization function to plot the background of each frame
    line.set_data([], [])
    dots.set_offsets( []  )
    return (line, dots, )

def animate(i): 
    # for animation: update line and scatter plots
    x_vec = np.multiply( l_vec[:,0], np.sin(y[i,:n]))
    y_vec = np.multiply( l_vec[:,0], np.cos(y[i,:n]))  
    x_pos = np.array( np.hstack([0, np.cumsum(x_vec)]) )
    y_pos = np.array( np.hstack([0, np.cumsum(y_vec)]))
    line.set_data( x_pos,y_pos ) 
    dots.set_offsets( np.vstack( (x_pos,y_pos )  ).transpose().tolist()  )  
    return (line,dots, ) 

def animate_pendulum(fig, animate, init, n_frames, fps, line, dots):
    #  call the animator.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
           frames=n_frames, interval= int(1000/fps), blit=True)
    html = HTML(anim.to_html5_video())
    plt.clf()
    plt.close('all')
    return(html)

In [11]:
# simulate pendulum dynamics 
y = odeint(pendulum_dynamics, y0, tInt) 

# Create figure and axis
fig = plt.figure(figsize=(fig_h, fig_v))
ax = fig.add_subplot(1, 1, 1)

# find vector of every pendulum, cumulative sum gives the position of every pendulum point 
x_vec = np.multiply(l_vec[:,0], np.sin(y0[:n]))
y_vec = np.multiply(l_vec[:,0], np.cos(y0[:n])) 
x_pos = np.array( np.hstack([0, np.cumsum(x_vec)]) )
y_pos = np.array( np.hstack([0, np.cumsum(y_vec)]))

# plot the first window, we will overwrite line and dots 
plt.plot([-1,1],[0,0],'k', zorder = -3) 
line = plt.plot(x_pos, y_pos)[0]
dots = plt.scatter(x_pos, y_pos, s=m*dot_scale)

# set plot parameters 
ax.set_xlim(-window, window)
ax.set_ylim(-window, window)
ax.set_aspect('equal')
ax.set_xticks([])
ax.set_yticks([])

html = animate_pendulum(fig, animate, init, n_frames, fps, line, dots)
display(html)
display(html)